In [221]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [222]:
product_type = "frozen"
num_time_periods = 7

In [223]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [224]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [225]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [226]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [227]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [228]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [229]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()
demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()

In [246]:
shelf_life['468a73f3'] = 3
cu_per_tu['468a73f3'] = 2
for key in demand['468a73f3'].keys():
    demand['468a73f3'][key] = 0
demand['468a73f3'][time_periods[1]] = 5
demand['468a73f3'][time_periods[3]] = 6
demand['468a73f3']

{'2022-06-13': 0,
 '2022-06-14': 5,
 '2022-06-15': 0,
 '2022-06-16': 6,
 '2022-06-17': 0,
 '2022-06-18': 0}

In [247]:
time_periods = forecast_data['DATE'].unique()

In [248]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_periods, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Xit1t2
orders_disaggregated = m.addVars(items, time_periods, time_periods, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, time_period] for item in items for time_period in time_periods)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[time_period] for time_period in time_periods)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, time_period] for item in items for time_period in time_periods)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for time_period_iterator, time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr((orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period] - clearances[item, time_period] >=  storage_used[item, time_periods[time_period_iterator]])
        else:
            m.addConstr((orders[item, time_period] * cu_per_tu[item]) + storage_used[item, time_periods[time_period_iterator-1]] 
            - demand[item][time_period] - clearances[item, time_period] >=  storage_used[item, time_periods[time_period_iterator]])

# inventory volume constraint
for time_period_iterator, time_period in enumerate(time_periods):
    if(time_period_iterator==0):
        m.addConstr(gp.quicksum(volume_per_cu[item] * ((cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])
    else:
        m.addConstr(gp.quicksum(volume_per_cu[item] * (storage_used[item, time_periods[time_period_iterator-1]] + (cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])

# # min/max constraints (linking too)
for item in demand.keys():
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= minimum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # if((not math.isnan(maximum_order_quantity_tu[item]))):
        #     m.addConstr(orders[item, time_period] <= maximum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # else:
        #     m.addConstr(orders[item, time_period] <= default_max_order * ordered_boolean[item, time_period])

for item in demand.keys():
    for time_period_iterator, time_period in enumerate(time_periods):
        life = shelf_life[item]
        if time_period_iterator >= life:
            m.addConstr(clearances[item, time_period] >= gp.quicksum((cu_per_tu[item] * (orders[item, t1]))  for t1 in time_periods[0:time_period_iterator-life])
            - gp.quicksum(demand[item][t1] for t1 in time_periods[0:time_period_iterator]) - gp.quicksum(clearances[item,t1] for t1 in time_periods[0:time_period_iterator-1]))

# # current inventory constraint
# for item in demand.keys():
#     for time_period_iterator,time_period in enumerate(time_periods):
#         if(time_period_iterator==0):
#             m.addConstr(storage_used[item, time_periods[time_period_iterator]] == (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])
#         else:
#             m.addConstr(storage_used[item, time_periods[time_period_iterator]] == storage_used[item, time_periods[time_period_iterator-1]] + (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])

# # clearance constraint
# for item in demand.keys():
#     life = shelf_life[item]
#     for time_period in time_periods[:-life]:
#         m.addConstr(clearances[item, time_period] <= (orders[item, time_period] * cu_per_tu[item]) - (gp.quicksum(orders_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:time_period_iterator+life])))



m.setParam(gp.GRB.Param.Heuristics, 0) 
m.optimize()

Set parameter Heuristics to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4197 rows, 25146 columns and 15003 nonzeros
Model fingerprint: 0x9562d99c
Variable types: 0 continuous, 25146 integer (2514 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+01]
  Objective range  [1e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 2434 rows and 21693 columns
Presolve time: 0.03s
Presolved: 1763 rows, 3453 columns, 8289 nonzeros
Variable types: 0 continuous, 3453 integer (416 binary)

Root relaxation: objective 4.654261e+03, 2247 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4654.26078    0  672          - 4654.26078      -     -    0s
 

In [249]:

for time_period in time_periods:
      print(clearances['468a73f3',time_period])

<gurobi.Var C7548 (value 1.0)>
<gurobi.Var C7549 (value -0.0)>
<gurobi.Var C7550 (value -0.0)>
<gurobi.Var C7551 (value -0.0)>
<gurobi.Var C7552 (value 0.0)>
<gurobi.Var C7553 (value 0.0)>


In [250]:
print(orders[('468a73f3',time_periods[0])])
print(demand['468a73f3'][time_periods[0]])
print(orders[('468a73f3',time_periods[1])])
print(demand['468a73f3'][time_periods[1]])
print(orders[('468a73f3',time_periods[2])])
print(demand['468a73f3'][time_periods[2]])
print(orders[('468a73f3',time_periods[3])])
print(demand['468a73f3'][time_periods[3]])
print(orders[('468a73f3',time_periods[4])])
print(demand['468a73f3'][time_periods[4]])
print(orders[('468a73f3',time_periods[5])])
print(demand['468a73f3'][time_periods[5]])


<gurobi.Var C0 (value 3.0)>
0
<gurobi.Var C1 (value -0.0)>
5
<gurobi.Var C2 (value 3.0)>
0
<gurobi.Var C3 (value -0.0)>
6
<gurobi.Var C4 (value -0.0)>
0
<gurobi.Var C5 (value -0.0)>
0
